# Lesson 1 – Single Node Pattern: Email Subject Booster Agent

This notebook is part of the **LangGraph Agentic AI – Intro Course**.

In this lesson, you will build a minimal LangGraph app: a single-node agent that improves email subject lines.


## 1. Objectives & Prerequisites

**Objectives**

By the end of this lesson, you can:

- Explain what a **state** and a **node** are in LangGraph.
- Build a **single-node graph** that transforms an email subject line into a slightly improved subject.
- Run the graph and inspect the resulting state.

**Prerequisites**

- Basic Python (functions, dicts, types).
- Very high-level understanding of “graphs” (nodes + edges) is helpful but not required.


## 2. Environment Setup

Make sure you have installed the course dependencies (from the repo root):

```bash
python -m venv .venv
source .venv/bin/activate   # on Windows: .venv\Scripts\activate
pip install -r env/requirements.txt
```

Then start Jupyter and select the `langgraph-intro` (or equivalent) Python kernel.


## 3. Concept Warm-up

**Mental model:**

- **State** – a Python object (we'll start with a `TypedDict`) that holds information flowing through the graph.
- **Node** – a function that:
  - takes the current state,
  - reads some fields,
  - writes some fields,
  - returns the new state.
- **Graph** – a set of nodes wired together with edges.  
  In this lesson, we will create **the smallest possible graph**: one node, no branching.

**Toy use case:**

We have a boring subject line like:

> `"meeting"`

We want the agent to transform it into something a bit clearer, like:

> `"Update: Meeting"` or `"Project Update – meeting"`


### Quick sanity check (optional)

Use this cell to run quick Python experiments while you follow along.


In [ ]:
# You can use this cell for any quick experiments while learning.

1 + 1

## 4. Define the State

We'll represent the state for this lesson as a simple `TypedDict` with two fields:

- `original_subject`: the input subject line.
- `improved_subject`: the subject line after our agent processes it.


In [1]:
from typing import TypedDict

class SubjectState(TypedDict):
    original_subject: str
    improved_subject: str

# Example of an initial state (we'll pass something like this into the graph later)
initial_state: SubjectState = {
    "original_subject": "meeting",
    "improved_subject": ""  # empty for now; the node will fill this in
}

initial_state

{'original_subject': 'meeting', 'improved_subject': ''}

## 5. Implement the Node Function

Now we define a node that:

1. Reads `original_subject` from the state.
2. Applies a couple of simple rules.
3. Writes `improved_subject` back into the state.
4. Returns the updated state.

Rules (for now):

- Trim leading/trailing spaces.
- If the subject is very short, prepend `"Update: "` and capitalize the first letter.
- Otherwise, just capitalize the first letter.


In [2]:
def improve_subject(state: SubjectState) -> SubjectState:
    """Simple subject booster node.
    
    - Trims whitespace
    - If very short, prepends 'Update: '
    - Ensures the first letter is capitalized
    """
    original = state["original_subject"].strip()
    
    if not original:
        improved = "No subject provided"
    elif len(original) < 10:
        improved = f"Update: {original.capitalize()}"
    else:
        improved = original[0].upper() + original[1:]
    
    state["improved_subject"] = improved
    return state


# Quick local test of the function without LangGraph yet
test_state: SubjectState = {"original_subject": "meeting", "improved_subject": ""}
improve_subject(test_state)

{'original_subject': 'meeting', 'improved_subject': 'Update: Meeting'}

## 6. Build and Run the LangGraph App

Now we wrap the node function into a LangGraph `StateGraph`.

Steps:

1. Create a `StateGraph` parametrized by `SubjectState`.
2. Add the `improve_subject` node.
3. Set the entry and finish points to this node.
4. Compile the graph into an executable app.
5. Invoke the app with an initial state.


In [3]:
from langgraph.graph import StateGraph

# 1. Create the graph
graph = StateGraph(SubjectState)

# 2. Add the node
graph.add_node("improve_subject", improve_subject)

# 3. Set entry and finish points
graph.set_entry_point("improve_subject")
graph.set_finish_point("improve_subject")  # single-node graph

# 4. Compile the graph
app = graph.compile()

# 5. Invoke with an initial state
result = app.invoke({"original_subject": "meeting", "improved_subject": ""})
result

{'original_subject': 'meeting', 'improved_subject': 'Update: Meeting'}

Try changing the `original_subject` and re-running the cell to see how the output changes.

For example:
- `"deadline tomorrow"`
- `" invoice 1234 "`
- `""` (empty string)


## 7. Exercises

Use the cells below to implement the exercises described in the README.

---

### Exercise 1 – Shortener vs Booster

**Goal:**  
Add a `mode: str` field (`"boost"` vs `"shorten"`) and modify `improve_subject` so that:

- If `mode == "shorten"` and the subject is longer than 40 characters:
  - Truncate and add `"..."`

Start by extending `SubjectState` and then update the node implementation.


In [4]:
# TODO: Extend SubjectState with a 'mode' field and update the node behavior.

# Hint:
# class SubjectState(TypedDict):
#     original_subject: str
#     improved_subject: str
#     mode: str  # "boost" or "shorten"



### Exercise 2 – Prefix by Category

**Goal:**  
Add a `category: str` field (e.g. `"project"`, `"invoice"`) and modify the node so that:

- If `category == "invoice"` → prefix with `"Invoice – "`
- You can define additional categories if you like.

Tip: apply category prefix logic **before** short/boost logic, or vice versa – but be consistent.


In [5]:
# TODO: Add category-based prefixing to the subject.

# Example idea:
# if state.get("category") == "invoice":
#     improved = f"Invoice – {improved}"



### Stretch Exercise 1 – Emoji Enhancer

**Goal:**  
Add emojis for certain keywords in the subject, for example:

- `"deadline"` → ⏰
- `"invoice"` → 💸

You can either:
- Append emojis to the end, or
- Insert them at the beginning.

Be careful not to add the same emoji multiple times if the function runs more than once.


In [ ]:
# TODO: Add emoji rules into your node.

# Example idea:
# if "deadline" in improved.lower():
#     improved += " ⏰"



### Stretch Exercise 2 – Validation

**Goal:**  
If `original_subject` is empty (or only whitespace), set:

```python
improved_subject = "No subject provided"
```

Make sure this check happens **before** any other transformation logic.


In [ ]:
# TODO: Ensure that empty/whitespace-only subjects are handled gracefully.

# Hint: you already saw a simple version of this logic in the first implementation
# of improve_subject. Tighten or extend that logic here.



---

You’ve now completed the scaffold for **Lesson 1 – Single Node Pattern**.

Next steps (outside this notebook):

- Commit this notebook and the README to your Git repo.
- Repeat similar patterns for Lessons 2–5.
- Later, we will build a larger **capstone agent** that reuses these ideas.
